In [1]:
%load_ext rpy2.ipython

from fig_builder import (
    TwinsFiguresData,
    TwinsFigures,
    condense_and_label_organism_positions,
)
from random import shuffle
from plotnine import *
import pandas as pd
import warnings
from glob import glob
from os.path import isfile
from os import environ

warnings.filterwarnings('ignore')

fig_data = TwinsFiguresData()
figs = TwinsFigures(fig_data)
ORGANISM = 'Chryseobacterium_indoltheticum'
SPARSE_N = 100
environ['CAP2_TMP_DIR'] = '/home/dcdanko/Data/cap_tmp'

In [2]:
gs = figs.snp_graph('Fusobacterium_necrophorum')
gs

processed 011515_TW_B /tmp/011515_TW_B.cap2__experimental__align_to_genome.bam__Fusobacterium_necrophorum.bam
processed 011515_TW_S /tmp/011515_TW_S.cap2__experimental__align_to_genome.bam__Fusobacterium_necrophorum.bam
processed 012015_HR_B /tmp/012015_HR_B.cap2__experimental__align_to_genome.bam__Fusobacterium_necrophorum.bam
processed 012015_HR_S /tmp/012015_HR_S.cap2__experimental__align_to_genome.bam__Fusobacterium_necrophorum.bam
processed 012016_HR_B /tmp/012016_HR_B.cap2__experimental__align_to_genome.bam__Fusobacterium_necrophorum.bam
processed 012016_HR_S /tmp/012016_HR_S.cap2__experimental__align_to_genome.bam__Fusobacterium_necrophorum.bam
processed 012715_TW_B /tmp/012715_TW_B.cap2__experimental__align_to_genome.bam__Fusobacterium_necrophorum.bam
processed 012715_TW_S /tmp/012715_TW_S.cap2__experimental__align_to_genome.bam__Fusobacterium_necrophorum.bam
processed 030116_TW_B /tmp/030116_TW_B.cap2__experimental__align_to_genome.bam__Fusobacterium_necrophorum.bam
processed 

{'TW': <networkx.classes.graph.Graph at 0x7f0451d226a0>,
 'HR': <networkx.classes.graph.Graph at 0x7f03f0a96a58>,
 'ISS': <networkx.classes.graph.Graph at 0x7f045042a710>}

In [3]:

figs.snp_graph('Fusobacterium_necrophorum')

{'TW': <networkx.classes.graph.Graph at 0x7f0450431198>,
 'ISS': <networkx.classes.graph.Graph at 0x7f035bfa3be0>}

In [ ]:
organisms = []
with open('strain_list.txt') as f:
    for line in f:
        organisms.append(line.strip().replace(' ', '_'))
        
for organism in organisms:
    figs.snp_graph(organism)

processed 011515_TW_B /tmp/011515_TW_B.cap2__experimental__align_to_genome.bam__Acholeplasma_axanthum.bam
processed 011515_TW_S /tmp/011515_TW_S.cap2__experimental__align_to_genome.bam__Acholeplasma_axanthum.bam
processed 012015_HR_B /tmp/012015_HR_B.cap2__experimental__align_to_genome.bam__Acholeplasma_axanthum.bam
processed 012015_HR_S /tmp/012015_HR_S.cap2__experimental__align_to_genome.bam__Acholeplasma_axanthum.bam
processed 012016_HR_B /tmp/012016_HR_B.cap2__experimental__align_to_genome.bam__Acholeplasma_axanthum.bam
processed 012016_HR_S /tmp/012016_HR_S.cap2__experimental__align_to_genome.bam__Acholeplasma_axanthum.bam
processed 012715_TW_B /tmp/012715_TW_B.cap2__experimental__align_to_genome.bam__Acholeplasma_axanthum.bam
processed 012715_TW_S /tmp/012715_TW_S.cap2__experimental__align_to_genome.bam__Acholeplasma_axanthum.bam
processed 030116_TW_B /tmp/030116_TW_B.cap2__experimental__align_to_genome.bam__Acholeplasma_axanthum.bam
processed 030116_TW_S /tmp/030116_TW_S.cap2__e

In [ ]:
def get_node_weights(G):
    node_weights = {}
    for node in G.nodes():
        total_weight = 0
        for el in G.edges(node, data=True):
            total_weight += el[2]['weight']
        node_weights[node] = total_weight
    return node_weights


def sparse_filter_graph(G, contig=None, coords=None):
    G = G.copy()
    node_weights = get_node_weights(G)
    for node, w in node_weights.items():
        if w == 1:  # never accept a SNP that we only saw once
            G.remove_node(node)
        ((seq, coord), miss) = node
        if contig and seq != contig:
            G.remove_node(node)
        if coords and coord < coords[0]:
            G.remove_node(node)
        if coords and coord > coords[1]:
            G.remove_node(node)
    return G

def sparse_partition(G, contig=None, coords=None):
    G = sparse_filter_graph(G, contig=contig, coords=coords)
    partition = community.best_partition(G)
    weights = get_node_weights(G)
    out = {}
    for k, v in partition.items():
        if weights[k] < 50:
            continue
        out[k] = (v, weights[k])
    return out

In [ ]:

graphs = {}

def process_file(filename):
    sname = filename.split('/')[-1].split('.')[0]
    subject = get_time_label(sname)[0]
    graphs[subject] = graphs.get(subject, nx.Graph())
    graph = graphs[subject]
    bam = pysam.AlignmentFile(filename, 'rb')
    build_graph(bam, G=graph)

for bam in glob('strains/serratia_proteamaculans/*.bam'):
    print(bam)
    process_file(bam)
    
graphs